  # MISC

  Here some usefull functions for plotting

In [ ]:
from scipy.ndimage import gaussian_filter
import mediapipe as mp
from sklearn.metrics import euclidean_distances
import cv2
import numpy as np
# compute the euclidean distances between all landmarks
# this script is used by other functions
image_file_name ="/home/frea/Documents/pyVHRnew/img/face.png"
landmarks_list =  [_ for _ in range(468)]
imag = cv2.imread(image_file_name,cv2.COLOR_RGB2BGR)
imag = cv2.cvtColor(imag, cv2.COLOR_BGR2RGB)
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    min_detection_confidence=0.5) as face_mesh:
  for idx, file in enumerate([image_file_name,]):
    image = cv2.imread(file)
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if not results.multi_face_landmarks:
      continue
    width = image.shape[1]
    height = image.shape[0]
    face_landmarks = results.multi_face_landmarks[0]
    ldmks = np.zeros((468, 3), dtype=np.float32)
    for idx, landmark in enumerate(face_landmarks.landmark):
        if ((landmark.HasField('visibility') and landmark.visibility < VISIBILITY_THRESHOLD)
                or (landmark.HasField('presence') and landmark.presence < PRESENCE_THRESHOLD)):
            ldmks[idx, 0] = -1.0
            ldmks[idx, 1] = -1.0
            ldmks[idx, 2] = -1.0
        else:
            coords = mp_drawing._normalized_to_pixel_coordinates(
                landmark.x, landmark.y, width, height)
            if coords:
                ldmks[idx, 0] = coords[1]
                ldmks[idx, 1] = coords[0]
                ldmks[idx, 2] = idx
            else:
                ldmks[idx, 0] = -1.0
                ldmks[idx, 1] = -1.0
                ldmks[idx, 2] = -1.0
euclid_distances = euclidean_distances(ldmks[:,0:2], ldmks[:,0:2])

In [ ]:
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
from numba import njit, prange
import plotly.graph_objects as go
import plotly.express as px
from sklearn.metrics import euclidean_distances

@njit(parallel=True)
def __sum_images(a,b,alpha):
  for x in prange(a.shape[0]):
    for y in prange(a.shape[1]):
      a[x,y,0] = min((1-alpha)*a[x,y,0] + alpha*b[x,y,0],255)
      a[x,y,1] = min((1-alpha)*a[x,y,1] + alpha*b[x,y,1],255)
      a[x,y,2] = min((1-alpha)*a[x,y,2] + alpha*b[x,y,2],255)
  return a

def plot_heat_map(err, err_sorted, radius, sigma, alpha, reverse=True):
  configure_plotly_browser_state()
  image_file_name = "/home/frea/Documents/pyVHRnew/img/face.png"
  imag = cv2.imread(image_file_name,cv2.COLOR_RGB2BGR)
  imag = cv2.cvtColor(imag, cv2.COLOR_BGR2RGB)
  width = imag.shape[1]
  height = imag.shape[0]
  mp_drawing = mp.solutions.drawing_utils
  mp_face_mesh = mp.solutions.face_mesh
  with mp_face_mesh.FaceMesh(
      static_image_mode=True,
      max_num_faces=1,
      min_detection_confidence=0.5) as face_mesh:
    choosenldmks = [_ for _ in range(468)]
    ldmks = np.zeros((468, 5), dtype=np.float32)
    ldmks[:, 0] = -1.0
    ldmks[:, 1] = -1.0
    # face landmarks
    results = face_mesh.process(imag)
    if results.multi_face_landmarks:
      face_landmarks = results.multi_face_landmarks[0]
      landmarks = [l for l in face_landmarks.landmark]
      for idx in choosenldmks:
          landmark = landmarks[idx]
          if not ((landmark.HasField('visibility') and landmark.visibility < VISIBILITY_THRESHOLD)
                  or (landmark.HasField('presence') and landmark.presence < PRESENCE_THRESHOLD)):
              coords = mp_drawing._normalized_to_pixel_coordinates(
                  landmark.x, landmark.y, width, height)
              if coords:
                  ldmks[idx, 0] = coords[1]
                  ldmks[idx, 1] = coords[0]
  to_keep = [err_sorted[0],]
  for e in err_sorted[1:]:
    if np.min(euclid_distances[e,to_keep]) > radius:
      to_keep.append(e)
  to_keep = list(set(to_keep))
  filtered_ldmks = []
  for idx in to_keep:
      filtered_ldmks.append([ldmks[idx,0],ldmks[idx,1],ldmks[idx,2]])
  filtered_ldmks = np.array(filtered_ldmks, dtype=np.float32)
  grid = np.zeros((height, width,1),dtype=np.float32)
  _err = err
  if reverse: 
    _err = 1.0 / err
  for e in filtered_ldmks:
    grid[int(e[0]),int(e[1]),0] = _err[int(e[2])]
  Z = gaussian_filter(grid, sigma=sigma)
  Z = (Z/np.max(Z)) * 255
  Z = Z.astype(np.uint8)
  heat_map = cv2.applyColorMap(Z[:,:,0],cv2.COLORMAP_RAINBOW) 
  __sum_images(imag, heat_map, alpha)
  fig = px.imshow(imag)
  for l in filtered_ldmks:
      name = 'landmark_' + str(int(l[2]))
      fig.add_trace(go.Scatter(x=(l[1],), y=(l[0],),mode='markers',marker_color='rgba(0,0,255,1.0)', marker=dict(color='LightSkyBlue',size=2,),name=name))
  fig.show()

In [ ]:
# Compute equidistant landmarks given a radius and a starting seed
def equidistant_landmarks(seed, radius):
  to_keep = seed
  for e in range(0,468):
    if np.min(euclid_distances[e,to_keep]) > radius:
      to_keep.append(e)
  to_keep = list(set(to_keep))
  return to_keep

from pyVHR.plot.visualize import *
eqd_ldmks = equidistant_landmarks([4,], 24.0*2)
print(eqd_ldmks)
print(len(eqd_ldmks))
visualize_landmarks_list("/home/frea/Documents/pyVHRnew/img/face.png", eqd_ldmks)

# Extraction routine

In [ ]:
from pyVHR.extraction.sig_processing import *
from pyVHR.BVP.BVP import *
from pyVHR.BPM.BPM import *
from pyVHR.BVP.methods import *
from pyVHR.BVP.filters import *
from pyVHR.plot.visualize import *

VisualizeParams.renderer = 'colab'

In [ ]:
from pyVHR.datasets.dataset import datasetFactory
from pyVHR.utils.errors import *

dataset = datasetFactory(
    "lgi_ppgi", videodataDIR="/var/datasets/VHR/", BVPdataDIR="/var/datasets/VHR/")
dataset.loadFilenames()
allvideo = dataset.videoFilenames
for v in range(len(allvideo)):
  print(v, allvideo[v])

In [ ]:
wsize = 8 #seconds

In [ ]:
video_idx = 1
fname = dataset.getSigFilename(video_idx)
sigGT = dataset.readSigfile(fname)
bpmGT, timesGT = sigGT.getBPM(wsize)
videoFileName = dataset.getVideoFilename(video_idx)
print(videoFileName)
fps = get_fps(videoFileName)
print(fps)

In [ ]:
sig_extractor = SignalProcessing()
sig_extractor.display_cuda_device()
sig_extractor.choose_cuda_device(0)

In [ ]:
sig_extractor.set_skin_extractor(SkinExtractionConvexHull('GPU'))
#sig_extractor.set_skin_extractor(SkinExtractionFaceParsing('CPU'))

In [ ]:
sig_extractor.set_total_frames(10*fps)
# To precess the whole video pass 0

In [ ]:
SkinProcessingParams.RGB_LOW_TH = 55
SkinProcessingParams.RGB_HIGH_TH = 200

SignalProcessingParams.RGB_LOW_TH = 55
SignalProcessingParams.RGB_HIGH_TH = 200

In [ ]:
sig_extractor.set_visualize_skin_and_landmarks(
    visualize_skin=False, visualize_landmarks=False, visualize_landmarks_number=False, visualize_patch=False)

In [ ]:
sig_extractor.set_visualize_skin_and_landmarks(
    visualize_skin=True, visualize_landmarks=True, visualize_landmarks_number=True, visualize_patch=True)

In [ ]:
visualize_landmarks_list("/home/frea/Documents/pyVHRnew/img/face.png", [i for i in range(0,468)])

In [ ]:
filter_landmarks = [i for i in range(0,468)]
filter_landmarks = list(dict.fromkeys(filter_landmarks))
print(len(filter_landmarks))
visualize_landmarks_list("/home/frea/Documents/pyVHRnew/img/face.png", filter_landmarks)

In [ ]:
sig_extractor.set_landmarks(filter_landmarks)

In [ ]:
# holistic extraction
hol_sig = sig_extractor.extract_holistic(videoFileName, 1.0)

In [ ]:
# HOLISTIC analysis
windowed_hol_sig, hol_timesES = sig_windowing(hol_sig, wsize, 1, fps)
windowed_hol_sig = apply_filter(windowed_hol_sig,  BPfilter, params={'minHz':0.65, 'maxHz':4.0, 'fps':fps, 'order':6})
hol_bvps = RGB_sig_to_BVP(windowed_hol_sig, fps, device_type='cuda', method=cupy_CHROM)
hol_bvps = apply_filter(hol_bvps, BPfilter, params={'minHz':0.65, 'maxHz':4.0, 'fps':fps, 'order':6})
hol_bpmES = BVP_to_BPM(hol_bvps, fps)

In [ ]:
# SQUARE EXTRACTION
sig_extractor.set_square_patches_side(24.0)
sig = sig_extractor.extract_patches(videoFileName, 1.0,"squares", "mean")

In [ ]:
sig.shape

In [ ]:
visualize_skin_coll = sig_extractor.get_visualize_skin()
visualize_patches_coll = sig_extractor.get_visualize_patches()
print(len(visualize_skin_coll))
print(len(visualize_patches_coll))

In [ ]:
interactive_image_plot(visualize_patches_coll)

In [ ]:
# [usage] sig_windowing(sig, wsize, stride, fps)
windowed_sig, timesES = sig_windowing(sig, wsize, 1, fps)
print(len(windowed_sig))
print(windowed_sig[0].shape)

In [ ]:
visualize_windowed_sig(windowed_sig,0)

In [ ]:
bvps = RGB_sig_to_BVP(windowed_sig, fps, device_type='cuda', method=cupy_CHROM)

In [ ]:
print(len(bvps))
print(bvps[1].shape)

In [ ]:
bpmES = BVP_to_BPM(bvps, fps)

In [ ]:
bpmES[1].shape

In [ ]:
median_bpmES = multi_est_BPM_median(bpmES)

In [ ]:
visualize_BPMs([[median_bpmES, timesES, "medianES"], [hol_bpmES, hol_timesES, "holistic"], [bpmGT, timesGT, "GT"]])

In [ ]:
RMSE, MAE, MAX, PCC = getErrors(np.expand_dims(median_bpmES, axis=0), bpmGT, hol_timesES, timesGT)
printErrors(RMSE, MAE, MAX, PCC)

In [ ]:
RMSE, MAE, MAX, PCC = getErrors(np.expand_dims(hol_bpmES, axis=0), bpmGT, hol_timesES, timesGT)
printErrors(RMSE, MAE, MAX, PCC)

# LANDMARKS errors

In [ ]:
ldmks_bpm = np.zeros((sig.shape[1],len(bpmES)),np.float32)
for i in range(ldmks_bpm.shape[0]):
  for j in range(ldmks_bpm.shape[1]):
    ldmks_bpm[i,j] = bpmES[j][i]

In [ ]:
# visualize all separated ldmks bpm
configure_plotly_browser_state()
fig = go.Figure()
name = "GT"
fig.add_trace(go.Scatter(x=timesGT, y=bpmGT,
                    mode='lines+markers',marker_color='rgba(255.0,0.0,0.0, 1.0)', name=name))
i = 1
for e in ldmks_bpm:
  name = "BPM_" + str(i)
  fig.add_trace(go.Scatter(x=timesES, y=e,
                      mode='lines+markers',marker_color='rgba('+str((i+23)*122%255)+', '+str(((i+1)*222)%255)+', '+str(((i+2)*233)%255)+', 1.0)', name=name))
  i+=1
fig.show()

In [ ]:
ldmks_err = np.zeros((sig.shape[1], 4),dtype=np.float32)
for i in range(ldmks_err.shape[0]):
  RMSE, MAE, MAX, PCC = getErrors(np.expand_dims(ldmks_bpm[i,:], axis=0), bpmGT, timesES, timesGT)
  ldmks_err[i,0] = RMSE
  ldmks_err[i,1] = MAE
  ldmks_err[i,2] = MAX
  if np.isnan(PCC):
    PCC = 0.0
  ldmks_err[i,3] = PCC

In [ ]:
top_n = 20
radius = 40*2

# RMSE heat map

In [ ]:
RMSE_errors = ldmks_err[:,0]
RMSE_errors = RMSE_errors/np.sum(RMSE_errors)
RMSE_errors = RMSE_errors/np.max(RMSE_errors)
RMSE_sorted_errors = np.argsort(RMSE_errors)
print(np.min(RMSE_errors))
print(np.max(RMSE_errors))
print(RMSE_sorted_errors[:top_n])
print(RMSE_errors[RMSE_sorted_errors[:top_n]])

In [ ]:
visualize_landmarks_list("/home/frea/Documents/pyVHRnew/img/face.png", RMSE_sorted_errors[:top_n])

In [ ]:
plot_heat_map(RMSE_errors,RMSE_sorted_errors, radius,radius, 0.5)

# MAE heat map

In [ ]:
MAE_errors = ldmks_err[:,1]
MAE_errors = MAE_errors/np.sum(MAE_errors)
MAE_errors = MAE_errors/np.max(MAE_errors)
MAE_sorted_errors = np.argsort(MAE_errors)
print(np.min(MAE_errors))
print(np.max(MAE_errors))
print(MAE_sorted_errors[:top_n])
print(MAE_errors[MAE_sorted_errors[:top_n]])

In [ ]:
visualize_landmarks_list("/home/frea/Documents/pyVHRnew/img/face.png", MAE_sorted_errors[:top_n])

In [ ]:
plot_heat_map(MAE_errors,MAE_sorted_errors, radius,radius, 0.5)

# MAX heat map

In [ ]:
MAX_errors = ldmks_err[:,2]
MAX_errors = MAX_errors/np.sum(MAX_errors)
MAX_errors = MAX_errors/np.max(MAX_errors)
MAX_sorted_errors = np.argsort(MAX_errors)
print(np.min(MAX_errors))
print(np.max(MAX_errors))
print(MAX_sorted_errors[:top_n])
print(MAX_errors[MAX_sorted_errors[:top_n]])

In [ ]:
visualize_landmarks_list("/home/frea/Documents/pyVHRnew/img/face.png", MAX_sorted_errors[:top_n])

In [ ]:
plot_heat_map(MAX_errors,MAX_sorted_errors, radius,radius, 0.5)

# PCC heat map

In [ ]:
PCC_errors = ldmks_err[:,3]
PCC_errors = PCC_errors+1.0
PCC_errors = PCC_errors/np.sum(PCC_errors)
PCC_errors = PCC_errors/np.max(PCC_errors)
PCC_sorted_errors = np.argsort(PCC_errors)[::-1]
print(np.min(PCC_errors))
print(np.max(PCC_errors))
print(PCC_sorted_errors[:top_n])
print(PCC_errors[PCC_sorted_errors[:top_n]])

In [ ]:
visualize_landmarks_list("/home/frea/Documents/pyVHRnew/img/face.png", PCC_sorted_errors[:top_n])

In [ ]:
plot_heat_map(PCC_errors,PCC_sorted_errors, radius, radius, 0.5, reverse=False)

# TOP landmarks

In [ ]:
# median of topn
list_to_plot = []
for e in RMSE_sorted_errors[:top_n]:
  list_to_plot.append(e)
for e in MAE_sorted_errors[:top_n]:
  list_to_plot.append(e)
for e in MAX_sorted_errors[:top_n]:
  list_to_plot.append(e)
for e in PCC_sorted_errors[:top_n]:
  list_to_plot.append(e)
list_to_plot = list(set(list_to_plot))
top_landmarks = [[ldmks_bpm[i]] for i in list_to_plot]
top_landmarks = np.array(top_landmarks)
top_landmarks = np.squeeze(top_landmarks,axis=1)
top_landmarks = np.median(top_landmarks, axis=0)
visualize_BPMs([[top_landmarks,timesES,"top_ldmks"],[hol_bpmES ,hol_timesES,"hol"], [bpmGT,timesGT,"GT"]])

In [ ]:
RMSE, MAE, MAX, PCC = getErrors(np.expand_dims(top_landmarks, axis=0), bpmGT, hol_timesES, timesGT)
printErrors(RMSE, MAE, MAX, PCC)

In [ ]:
RMSE, MAE, MAX, PCC = getErrors(np.expand_dims(hol_bpmES, axis=0), bpmGT, hol_timesES, timesGT)
printErrors(RMSE, MAE, MAX, PCC)

In [ ]:
#common idx 
best_rmse = RMSE_sorted_errors[:top_n]
best_mae = MAE_sorted_errors[:top_n]
best_max = MAX_sorted_errors[:top_n]
best_pcc = PCC_sorted_errors[:top_n]
top_intersection = set(best_rmse).intersection(set(best_mae).intersection(set(best_max).intersection(best_pcc)))
print(top_intersection)
print(len(top_intersection))

In [ ]:
visualize_landmarks_list("/home/frea/Documents/pyVHRnew/img/face.png", list(top_intersection))